# Notebook 3: Memoria Conversazionale con LCEL

**Obiettivo**: Implementare memoria breve e a lungo termine per un chatbot con LangChain 1.0+ usando LCEL (LangChain Expression Language)

**Nota**: Questo notebook usa l'API moderna LCEL di LangChain 1.0+, non le API legacy (ConversationChain, ConversationBufferMemory).

---
Questo snippet implementa un chatbot conversazionale con memoria usando LangChain e Ollama.
Il codice crea un'istanza di ChatOllama che si connette al modello locale llama3.2:3b, poi utilizza un dizionario Python (store) per memorizzare le cronologie delle conversazioni separate per session_id.

La funzione get_session_history gestisce il recupero o la creazione di nuove cronologie in memoria.
Il wrapper RunnableWithMessageHistory integra automaticamente la gestione della memoria nel modello, permettendo di mantenere il contesto conversazionale tra chiamate successive.

Nell'esempio pratico, il chatbot ricorda il nome dell'utente nella seconda domanda perché entrambe le interazioni utilizzano lo stesso session_id ("abcd"), consentendo al modello di accedere alla cronologia completa della conversazione memorizzata in memoria.

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Usa ChatOllama direttamente senza prompt template custom
llm = ChatOllama(model="llama3.2:3b", temperature=0.7)

store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# Wrappa direttamente il modello
chat = RunnableWithMessageHistory(
    llm,
    get_session_history
)

# Usa
response = chat.invoke(
    [{"role": "user", "content": "Ciao, mi chiamo Marco"}],
    config={"configurable": {"session_id": "abcd"}}
)
print(response.content)

response = chat.invoke(
    [{"role": "user", "content": "Qual è il mio nome?"}],
    config={"configurable": {"session_id": "abcd"}}
)
print(response.content)

Ciao Marco! Sono felice di conoscerti. Come posso aiutarti oggi? Vuoi parlare di qualcosa in particolare o vuoi semplicemente chiacchierare un po'?
Mi dispiace, ma non sono sicuro del tuo nome. Hai solo detto "Ciao Marco" all'inizio della nostra conversazione, quindi non so se sia il tuo vero nome o semplicemente un saluto. Se vuoi, puoi dirmi il tuo nome reale!


## 1. ChatMessageHistory - Memoria Breve Termine

In LCEL, usiamo `InMemoryChatMessageHistory` per memorizzare tutti i messaggi della conversazione.


In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 1. Inizializza il modello LLM locale
llm = ChatOllama(model="llama3.2:3b", temperature=0.1)

# 2. Crea lo store per memorizzare le sessioni
store = {}

def get_session_history(session_id: str):
    """Recupera o crea una nuova cronologia per la sessione"""
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 3. Crea il chatbot con memoria
chat = RunnableWithMessageHistory(
    llm,
    get_session_history
)

# 4. SESSIONE STUDENTE 1
print("=== CONVERSAZIONE STUDENTE 1 ===")
response1 = chat.invoke(
    [{"role": "user", "content": "Ciao, mi chiamo Luca e studio a Milano"}],
    config={"configurable": {"session_id": "studente_1"}}
)
print(f"AI: {response1.content}\n")

response2 = chat.invoke(
    [{"role": "user", "content": "Dove studio io?"}],
    config={"configurable": {"session_id": "studente_1"}}
)
print(f"AI: {response2.content}\n")

# 5. SESSIONE STUDENTE 2 (sessione diversa, memoria separata)
print("=== CONVERSAZIONE STUDENTE 2 ===")
response3 = chat.invoke(
    [{"role": "user", "content": "Ciao, mi chiamo Sara e studio a Roma"}],
    config={"configurable": {"session_id": "studente_2"}}
)
print(f"AI: {response3.content}\n")

response4 = chat.invoke(
    [{"role": "user", "content": "Dove studio io?"}],
    config={"configurable": {"session_id": "studente_2"}}
)
print(f"AI: {response4.content}\n")

# 6. Torniamo alla SESSIONE STUDENTE 1
print("=== TORNIAMO ALLO STUDENTE 1 ===")
response5 = chat.invoke(
    [{"role": "user", "content": "Come mi chiamo?"}],
    config={"configurable": {"session_id": "studente_1"}}
)
print(f"AI: {response5.content}")


## 2. Chat con Memoria Buffer usando LCEL

Questo pattern rappresenta l'implementazione production-ready di un chatbot conversazionale con memoria utilizzando l'architettura LCEL (LangChain Expression Language). È il metodo standard per costruire assistenti AI professionali con gestione della cronologia conversazionale.

### Architettura a Tre Livelli
Il codice separa chiaramente tre responsabilità. Il ChatPromptTemplate definisce la struttura dell'interazione, includendo un messaggio di sistema per configurare il comportamento dell'assistente, un MessagesPlaceholder per iniettare dinamicamente la cronologia conversazionale e un template per l'input utente. La chain LCEL (prompt | llm) crea una pipeline che processa sequenzialmente il prompt template e lo invia al modello. Il wrapper RunnableWithMessageHistory orchestra automaticamente il recupero della cronologia, l'integrazione nel prompt e il salvataggio dei nuovi messaggi.

### Gestione Avanzata degli Input
I parametri input_messages_key e history_messages_key sono necessari perché il runnable accetta un dizionario come input anziché una semplice lista. Questi specificano dove mappare l'input dell'utente e dove iniettare la cronologia nel template, consentendo a LangChain di gestire automaticamente il flusso dei dati tra memoria e modello.

### Vantaggi del Pattern
Questo approccio garantisce modularità e manutenibilità. Puoi modificare il comportamento del sistema cambiando solo il prompt template, aggiungere preprocessing/postprocessing estendendo la chain, o sostituire il backend di memoria senza riscrivere la logica applicativa. Il sistema supporta nativamente sessioni multiple isolate attraverso session_id, rendendolo ideale per applicazioni multi-utente.


In [ ]:
# Crea prompt template con LCEL (ChatPromptTemplate)
prompt = ChatPromptTemplate.from_messages([
    ("system", "Sei un assistente amichevole. Rispondi alle domande dell'utente."),
    MessagesPlaceholder(variable_name="history"),  # Placeholder per la storia
    ("human", "{input}")  # Input dell'utente
])

# Crea chain LCEL: prompt | llm
chain = prompt | llm

# Store per gestire memoria per diverse sessioni
store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    """Funzione per ottenere/creare memoria per una sessione"""
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# Crea RunnableWithMessageHistory (gestisce automaticamente la memoria)
conversation = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# Test conversazione (LCEL usa invoke con config)
session_id = "test_session"

print("=== Conversazione 1 ===")
risposta1 = conversation.invoke(
    {"input": "Ciao, mi chiamo Mario"},
    config={"configurable": {"session_id": session_id}}
)
print(f"Risposta: {risposta1.content}\n")

print("=== Conversazione 2 ===")
risposta2 = conversation.invoke(
    {"input": "Qual è il mio nome?"},
    config={"configurable": {"session_id": session_id}}
)
print(f"Risposta: {risposta2.content}\n")

print("=== Conversazione 3 ===")
risposta3 = conversation.invoke(
    {"input": "Dove lavoro?"},
    config={"configurable": {"session_id": session_id}}
)
print(f"Risposta: {risposta3.content}\n")


### Flusso di Esecuzione
Quando si invoca `conversation.invoke({"input": "Ciao, mi chiamo Mario"}, config={"configurable": {"session_id": "test_session"}})`, succede questo in sequenza:
  - LangChain chiama get_session_history("test_session") per recuperare la cronologia
  - sostituisce {input} nel template con "Ciao, mi chiamo Mario", inserisce i messaggi storici nel MessagesPlaceholder
  - invia tutto al modello tramite la chain
  - salva automaticamente sia il messaggio utente che la risposta AI nella memoria.



Nella seconda invocazione, quando chiedi "Qual è il mio nome?", il sistema recupera la stessa cronologia contenente il messaggio precedente, quindi il modello può rispondere correttamente "Mario".


## 3. Chat con Summary Memory

Usiamo summary memory in una conversazione lunga.

Questo codice implementa un sistema di summary memory incrementale che aggiorna progressivamente il riassunto della conversazione invece di ricrearlo da zero ogni volta.

### Logica del Summary Incrementale
Quando la conversazione supera 6 messaggi, il sistema recupera il summary precedente da `summary_text_store` e lo combina con i nuovi messaggi da archiviare.
Il prompt di aggiornamento chiede esplicitamente all'LLM di "combinare le vecchie info con le nuove", mantenendo i dati storici (nome, città iniziali) e aggiungendo i nuovi dettagli (hobby, interessi emersi dopo).

### Vantaggi dell'Approccio
Questo metodo preserva meglio le informazioni iniziali attraverso molteplici cicli di summarization. Il summary diventa cumulativo: ogni volta che viene aggiornato, porta con sé i fatti delle conversazioni precedenti.

### Flusso di Esecuzione
Il sistema conta i messaggi regolari escludendo il messaggio [CONTEXT] che contiene il summary. Quando supera il limite, mantiene gli ultimi 2 messaggi recenti per continuità e invia all'LLM sia il vecchio summary che i nuovi messaggi da archiviare. L'LLM genera un summary aggiornato che viene salvato in `summary_text_store` e iniettato come primo HumanMessage nella nuova cronologia.

La temperature ridotta a 0.1 garantisce risposte più deterministiche e accurate nel recupero delle informazioni.



In [2]:
from langchain_ollama import ChatOllama
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# Inizializza LLM
llm = ChatOllama(model="llama3.2:3b", temperature=0.1)

# Prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", """Sei un assistente amichevole.
IMPORTANTE: Leggi attentamente TUTTA la cronologia.
Se c'è un messaggio di [CONTEXT] o [RIASSUNTO], usalo come verità assoluta sull'utente."""),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

chain = prompt | llm

# Store
summary_store = {}
summary_text_store = {}

def get_summary_history(session_id: str, max_messages=6) -> BaseChatMessageHistory:
    """Crea/recupera cronologia con summary INCREMENTALE"""

    if session_id not in summary_store:
        summary_store[session_id] = InMemoryChatMessageHistory()
        summary_text_store[session_id] = "" # Inizializza stringa vuota

    history = summary_store[session_id]

    # 1. Identifica i messaggi "veri" (escludendo il messaggio tecnico di context se presente)
    regular_messages = [m for m in history.messages
                       if not (isinstance(m, HumanMessage) and m.content.startswith("[CONTEXT]"))]

    # 2. Controllo Limite
    if len(regular_messages) > max_messages:
        print(f"\n⚠️ LIMITE SUPERATO ({len(regular_messages)} messaggi). Aggiorno il summary...\n")

        # Separiamo messaggi da archiviare (old) e messaggi da tenere (recent)
        # Teniamo gli ultimi 2 per mantenere il flusso della conversazione fluido
        messages_to_summarize = regular_messages[:-2]
        recent_messages = regular_messages[-2:]

        # Recuperiamo il VECCHIO summary (se esiste)
        previous_summary = summary_text_store.get(session_id, "")

        # Convertiamo i messaggi da archiviare in testo
        conversation_text = "\n".join([
            f"{'UTENTE' if isinstance(m, HumanMessage) else 'AI'}: {m.content}"
            for m in messages_to_summarize
        ])

        # 3. Prompt di AGGIORNAMENTO (Cruciale: Unisce Vecchio + Nuovo)
        update_prompt = f"""Sei un gestore di memoria.
Ecco le informazioni che già conosci sull'utente:
{previous_summary}

Ecco la nuova conversazione appena avvenuta:
{conversation_text}

COMPITO:
Aggiorna il profilo utente combinando le vecchie info con le nuove.
Mantieni nome, città, lavoro e aggiungi nuovi interessi o dettagli.
Sii sintetico. Rispondi SOLO con la lista dei fatti aggiornata.
"""

        # Genera il nuovo summary
        summary_response = llm.invoke([{"role": "user", "content": update_prompt}])
        new_summary_text = summary_response.content.strip()

        # Salviamo il testo aggiornato
        summary_text_store[session_id] = new_summary_text
        print(f"📝 Summary Aggiornato: {new_summary_text}\n")

        # 4. Ricostruzione della History
        new_history = InMemoryChatMessageHistory()

        # Iniettiamo il summary come PRIMO messaggio (chiaro per l'LLM)
        # Usiamo un formato esplicito
        context_message = HumanMessage(content=f"[CONTEXT] RIEPILOGO DATI UTENTE:\n{new_summary_text}")
        new_history.add_message(context_message)

        # Reinseriamo gli ultimi messaggi per non perdere il filo immediato
        for msg in recent_messages:
            new_history.add_message(msg)

        # Sostituiamo la history nello store
        summary_store[session_id] = new_history
        return new_history

    return history

# Setup Runnable
conversation_summary = RunnableWithMessageHistory(
    chain,
    get_summary_history,
    input_messages_key="input",
    history_messages_key="history"
)

# --- TEST ---
print("=== TEST SUMMARY INCREMENTALE ===\n")
session_id = "test_incrementale_v2"

# Simuliamo una conversazione lunga per forzare il summary
messaggi = [
    "Ciao, mi chiamo Mario.",                       # 1
    "Lavoro a Palermo come sviluppatore.",          # 2
    "Ho 5 anni di esperienza in Python.",           # 3
    "Sto studiando LangChain.",                     # 4
    "Mi piace cucinare la pasta.",                  # 5
    "Voglio fare un chatbot.",                      # 6
    "Mi piace anche il calcio.",                    # 7
    "A che punto siamo?",                           # 8
    "Come mi chiamo e dove lavoro?",                # 9
    "Qual è il mio framework preferito?",           # 10
]

# messaggi = [
#     "Ciao, mi chiamo Giulia.",
#     "Attualmente abito a Milano.",
#     "Lavoro come Data Analyst in una banca.",
#     "Uso Python e SQL tutti i giorni per lavoro.",
#     "Nel tempo libero mi piace molto fare giardinaggio.",
#     "Ho un cane di nome Rex.",
#     "Il mio piatto preferito è la carbonara.",
#     "Vorrei imparare a usare LangChain per automatizzare dei report.",
#     "Qual è la differenza principale tra LangChain e LlamaIndex?",
#     "Sto avendo qualche difficoltà a capire come funzionano gli agenti.",
#     "Ieri sono andata al cinema a vedere un film di fantascienza.",
#     "Tra l'altro, il mio colore preferito è il verde.",
#     "Ho una notizia: tra un mese mi trasferirò a Torino.",
#     "Lì continuerò a lavorare ma in smart working.",
#     "Vorrei anche iscrivermi a un corso di tennis.",
#     "Non bevo caffè, preferisco il tè matcha.",
#     "Mi consigli un buon libro tecnico sull'AI?",
#     "Sto provando a far girare un modello locale sul mio PC.",
#     "Senti, facciamo un punto della situazione.",
#     "Ti ricordi come mi chiamo, dove vivo ora e dove andrò a vivere tra poco?"
# ]

for i, msg in enumerate(messaggi, 1):
    print(f"--- Turno {i} ---")
    print(f"👤 {msg}")

    risposta = conversation_summary.invoke(
        {"input": msg},
        config={"configurable": {"session_id": session_id}}
    )

    print(f"🤖 {risposta.content}\n")

    # Debug Memoria
    history = summary_store[session_id].messages
    if len(history) > 0 and "[CONTEXT]" in history[0].content:
        print(f"🔍 Contenuto Memoria (Primo Messaggio): {history[0].content}...")

=== TEST SUMMARY INCREMENTALE ===

--- Turno 1 ---
👤 Ciao, mi chiamo Mario.
🤖 Ciao Mario! Sono felice di conoscerti! Come posso aiutarti oggi?

--- Turno 2 ---
👤 Lavoro a Palermo come sviluppatore.
🤖 Ciao Mario! Sembra che tu sia un professionista molto specifico, lavorando come sviluppatore a Palermo. Quale tipo di progetti stai lavorando attualmente? Ecco, posso aiutarti con qualsiasi domanda o problema tecnico che possa avere.

--- Turno 3 ---
👤 Ho 5 anni di esperienza in Python.
🤖 Hai una solida base di conoscenza in Python! 5 anni di esperienza sono un ottimo risultato, dimostra la tua capacità di apprendimento e adattamento nel campo della programmazione.

Quale tipo di progetti hai lavorato fino a ora? Sono stati più progetti personali o hai avuto l'opportunità di lavorare su progetti aziendali?

--- Turno 4 ---
👤 Sto studiando LangChain.
🤖 LangChain è un framework molto interessante per la programmazione naturale e la gestione delle interazioni con i dati. È una tecnologia in c

## 4. Memoria Persistente su File

Salviamo la memoria su file per persistenza tra sessioni usando ChatMessageHistory.

Questo codice implementa la persistenza della memoria conversazionale su disco, permettendo di salvare e ricaricare la cronologia dei messaggi tra diverse sessioni.

### Funzione di Salvataggio
salva_memoria() converte l'oggetto InMemoryChatMessageHistory in formato JSON serializzabile. Itera su tutti i messaggi nella memoria, identifica il tipo (HumanMessage o AIMessage) e crea un dizionario con due campi: type (human/ai) e content (testo del messaggio). La lista di dizionari viene salvata in un file JSON con encoding UTF-8 e formattazione indentata per leggibilità.

### Funzione di Caricamento
carica_memoria() ricrea l'oggetto InMemoryChatMessageHistory dal file JSON. Prima controlla se il file esiste con os.path.exists(), poi legge il JSON e itera sui messaggi salvati. Per ogni messaggio, in base al campo type, aggiunge il contenuto alla memoria usando add_user_message() o add_ai_message() che creano automaticamente gli oggetti HumanMessage/AIMessage corretti.

### Caso d'Uso Pratico
Questo pattern è essenziale per applicazioni reali dove vuoi che gli utenti riprendano le conversazioni dopo aver chiuso l'applicazione. Invece di perdere tutto il contesto conversazionale quando il programma termina, puoi salvare la memoria prima della chiusura e ricaricarla all'avvio successivo. Nel tuo sistema di summary memory, potresti salvare sia summary_store che summary_text_store per preservare anche i riassunti generati.


In [ ]:
import os
import json
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import messages_to_dict, messages_from_dict, HumanMessage, AIMessage

def salva_memoria(memory: InMemoryChatMessageHistory, filepath="memoria_chat.json"):
    # Converte tutti i messaggi in dizionari standard LangChain
    dicts = messages_to_dict(memory.messages)
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(dicts, f, ensure_ascii=False, indent=2)
    print(f"✅ Memoria salvata (formato nativo) su {filepath}")

def carica_memoria(filepath="memoria_chat.json") -> InMemoryChatMessageHistory:
    memory = InMemoryChatMessageHistory()
    if os.path.exists(filepath):
        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)
            # Ricostruisce automaticamente gli oggetti corretti (Human, AI, System, Tool, etc.)
            messages = messages_from_dict(data)
            memory.add_messages(messages)
        print(f"✅ Memoria caricata ({len(messages)} messaggi)")
    return memory

# TEST
memoria = InMemoryChatMessageHistory()
memoria.add_user_message("Ciao")
memoria.add_ai_message("Ciao! Come posso aiutarti?")

# Salvataggio
salva_memoria(memoria)

# Caricamento
nuova_memoria = carica_memoria()
print(nuova_memoria.messages)

## 5. Esempio Pratico: Chatbot con Memoria Persistente

Creiamo un chatbot che ricorda informazioni tra sessioni usando LCEL e persistenza su file.


In [ ]:
import os
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

# Assumendo che le funzioni salva_memoria e carica_memoria siano definite nelle celle precedenti

class ChatbotConMemoria:
    """Chatbot con memoria persistente su JSON usando LCEL"""

    def __init__(self, llm, user_id="default", system_prompt="Sei un assistente utile."):
        self.user_id = user_id
        # Nome file dinamico basato sull'ID utente
        self.memory_file = f"memoria_{user_id}.json"
        self.llm = llm

        # 1. Carica memoria (o ne crea una nuova)
        self.memory = self._carica_memoria()

        # 2. Crea prompt template
        prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{input}")
        ])

        # 3. Crea chain base
        chain = prompt | self.llm

        # 4. Avvolge la chain con la gestione della memoria
        # Nota: Qui 'session_id' è richiesto dalla firma di RunnableWithMessageHistory,
        # ma noi forziamo l'uso di 'self.memory' perché questa classe gestisce un solo utente alla volta.
        self.chain_with_history = RunnableWithMessageHistory(
            chain,
            get_session_history=lambda session_id: self.memory,
            input_messages_key="input",
            history_messages_key="history"
        )

    def _carica_memoria(self) -> InMemoryChatMessageHistory:
        """Carica da file se esiste, altrimenti nuova memoria vuota"""
        if os.path.exists(self.memory_file):
            print(f"📂 Memoria trovata per utente: {self.user_id}")
            # Qui usiamo la tua funzione definita in precedenza
            return carica_memoria(self.memory_file)
        else:
            return InMemoryChatMessageHistory()

    def _salva_memoria(self):
        """Salva lo stato attuale della memoria su file"""
        # Qui usiamo la tua funzione definita in precedenza
        salva_memoria(self.memory, self.memory_file)

    def chat(self, messaggio):
        """Invia messaggio, ottieni risposta e salva"""
        # Il session_id qui è 'dummy' perché forziamo self.memory nel costruttore,
        # ma LangChain lo richiede comunque nella config.
        risposta = self.chain_with_history.invoke(
            {"input": messaggio},
            config={"configurable": {"session_id": self.user_id}}
        )

        # Salvataggio automatico dopo ogni interazione
        self._salva_memoria()

        return risposta.content

    def reset(self):
        """Cancella memoria e file"""
        self.memory.clear()
        if os.path.exists(self.memory_file):
            os.remove(self.memory_file)
        print(f"🗑️ Memoria resettata per {self.user_id}")

# --- ESECUZIONE TEST ---

# Definizione LLM (assicurati di averlo definito, es: OpenAI o ChatOllama)
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(model="gpt-3.5-turbo")

print("=== Test Chatbot Persistent ===\n")

# Sessione 1
bot_mario = ChatbotConMemoria(llm, user_id="mario", system_prompt="Sei un assistente siciliano simpatico.")
print(f"Bot: {bot_mario.chat('Ciao, sono Mario e adoro le arancine!')}")

# Sessione 2 (Simulazione riavvio script)
print("\n--- Simulazione riavvio script ---\n")
bot_mario_bis = ChatbotConMemoria(llm, user_id="mario") # Ricarica lo stesso file
print(f"Bot: {bot_mario_bis.chat('Cosa mi piace mangiare?')}")

## 6. Esercizio: Memoria Persistente con Summarization

### Obiettivo
Avete imparato a creare un chatbot che salva la conversazione su file JSON (persistenza) e uno che riassume i vecchi messaggi quando la memoria diventa troppo piena (summarization).

Ora dovete unire queste due funzionalità in un unico script. Il vostro compito è creare una classe o uno script che gestisca un chatbot capace di mantenere una conversazione infinita, ottimizzando i token tramite riassunto, e che sia in grado di "spegnersi" e "riaccendersi" senza perdere né i messaggi recenti né il riassunto accumulato.

### Requisiti Tecnici
Integrazione:
 - lo script deve utilizzare la logica di Summarization (aggiornamento del contesto quando si supera un limite N di messaggi) e salvare il tutto su disco.
 - Il File JSON: il salvataggio non riguarda più solo una lista di messaggi. Dovete salvare (e ricaricare) lo stato completo della memoria.
 - Suggerimento: riflettete su come salvare il "testo del riassunto" attuale insieme ai "messaggi recenti".

HINT: per testare il funzionamento, impostate il limite di messaggi nel buffer a un numero basso (es. 4 o 6 messaggi) in modo che il riassunto scatti velocemente.


## 7. Note e Best Practices

### Cosa abbiamo imparato:
1. **ChatMessageHistory**: Memorizzare tutti i messaggi usando l'API moderna di LangChain 1.0+
2. **RunnableWithMessageHistory**: Gestire automaticamente la memoria nelle chain
3. **Conversation Summary**: Riassumere i messaggi per gestire conversazioni lunghe e risparmiare token
4. **Memoria persistente**: Salvare su file per mantenere contesto tra le sessioni dell'utente

**LCEL (LangChain Expression Language)**: usare un approccio moderno e flessibile per creare chain (langchain 1.0+)

### Quando usare cosa:
- **ChatMessageHistory**: per brevi conversazioni (< 10 turni), prototipi, sviluppo
- **Conversation Summary**: per conversazioni lunghe (> 20 turni), produzione
- **Memoria persistente**: Quando serve ricordare tra sessioni o riavvii


### Prossimi passi:
- Aggiungeremo RAG per knowledge base
- Integreremo memoria con Agent e Tools

---

**Congratulazioni! Hai completato il Notebook 3! 🎉**
